# Training Model (DNN)

### Import 

In [2]:
import csv
import tensorflow as tf
import numpy as np

/home/sunset/.pyenv/versions/anaconda3-5.0.1/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Set Variable

In [3]:
data_x = []
data_y = []

test_x = []
test_y = []

train_x = tf.placeholder(tf.float32, shape=[None, 3])
train_y = tf.placeholder(tf.float32, shape=[None, 1])

### Read Dataset

In [4]:
def read_data():
    
    with open('dataset/athlete_events.csv', 'r', encoding='utf-8') as f:

        reader = csv.reader(f)

        for row in reader:
            
            tmp = []

            if row[3] == 'NA' or row[4] == 'NA' or row[5] == 'NA' or row[14] == 'NA':
                pass

            else:
                tmp.append(row[3])
                tmp.append(row[4])
                tmp.append(row[5])
                
                data_x.append(tmp)
                
                if row[14] == 'Gold':
                    data_y.append([1])
                elif row[14] == 'Silver':
                    data_y.append([2])
                elif row[14] == 'Bronze':
                    data_y.append([3])
                else:
                    data_y.append(row[14])
                

        del data_x[0]
        del data_y[0]
        
        global test_x
        test_x = data_x[29181:]
        del data_x[29181:]
    
        global test_y
        test_y = data_y[29181:]
        del data_y[29181:]

        x_shape = np.array(data_x)
        y_shape = np.array(data_y)

        tx_shape = np.array(test_x)
        ty_shape = np.array(test_y)

        print("data_X Shape : ", x_shape.shape)
        print("data_Y Shape : ", y_shape.shape)

        print("tdata_X Shape : ", tx_shape.shape)
        print("tdata_Y Shape : ", ty_shape.shape)


# def set_train_data():
    
#     datas = read_data()

#     for row in datas:
#         tmp = []

#         tmp.append(float(row['age']))
#         tmp.append(float(row['height']))
#         tmp.append(float(row['weight']))

#         data_x.append(tmp)

        

#     global test_x
#     test_x = data_x[29181:]
#     del data_x[29181:]
    
#     global test_y
#     test_y = data_y[29181:]
#     del data_y[29181:]

#     x_shape = np.array(data_x)
#     y_shape = np.array(data_y)

#     tx_shape = np.array(test_x)
#     ty_shape = np.array(test_y)

#     print("data_X Shape : ", x_shape.shape)
#     print("data_Y Shape : ", y_shape.shape)

#     print("tdata_X Shape : ", tx_shape.shape)
#     print("tdata_Y Shape : ", ty_shape.shape)

### Set Dataset

In [5]:
read_data()

data_X Shape :  (29181, 3)
data_Y Shape :  (29181, 1)
tdata_X Shape :  (1000, 3)
tdata_Y Shape :  (1000, 1)


### Set Model

In [7]:
w1 = tf.Variable(tf.random_normal([3, 5]))
b1 = tf.Variable(tf.random_normal([5]))
l1 = tf.nn.relu(tf.add(tf.matmul(train_x, w1), b1))

w2 = tf.Variable(tf.random_normal([5, 5]))
b2 = tf.Variable(tf.random_normal([5]))
l2 = tf.nn.relu(tf.add(tf.matmul(l1, w2), b2))

w3 = tf.Variable(tf.random_normal([5, 5]))
b3 = tf.Variable(tf.random_normal([5]))
l3 = tf.nn.relu(tf.add(tf.matmul(l2, w3), b3))

w4 = tf.Variable(tf.random_normal([5, 1]))
b4 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.add(tf.matmul(l3, w4), b4)

cost = tf.losses.mean_squared_error(labels=train_y, predictions=hypothesis)

learning_rate = 0.01

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### Set Saver

In [8]:
saver = tf.train.Saver()

save_path = "./checkpoints/olympic_nn_model.ckpt"

### Train

In [9]:
with tf.Session() as sess:
    
    # Initialize
    init = tf.global_variables_initializer()
    sess.run(init)

    # TensorBoard Setting
    cost_summ = tf.summary.scalar("cost", cost)
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./log')
    writer.add_graph(sess.graph)

    # Learning Start
    print("Learning Start!")
    for step in range(10001):
        cost_v, _ = sess.run([cost, train], feed_dict={train_x: data_x, train_y: data_y})
        if step % 200 == 0:
            print("STEP :", step, "Cost :", cost_v)
            
            
    # Model Save
    print("Saving Model...")        
    saver.save(sess, save_path)
    print("Save Successful!")

    # Model Testing
    print("Model Test...")
    result = sess.run([hypothesis], feed_dict={train_x: [[22.0, 185.0, 82.0]]})
    print(result)

    # Check Accuracy
    print("Checking Accuracy...")
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(train_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy :",sess.run(accuracy, feed_dict={train_x: test_x, train_y: test_y}) * 100,"%")

    print("Finished")

Learning Start!
STEP : 0 Cost : 767.45294
STEP : 200 Cost : 0.70070976
STEP : 400 Cost : 0.67703164
STEP : 600 Cost : 0.6729292
STEP : 800 Cost : 0.67259765
STEP : 1000 Cost : 0.67258275
STEP : 1200 Cost : 0.6725801
STEP : 1400 Cost : 0.6725768
STEP : 1600 Cost : 0.6725731
STEP : 1800 Cost : 0.6725691
STEP : 2000 Cost : 0.67256343
STEP : 2200 Cost : 0.6725578
STEP : 2400 Cost : 0.67255384
STEP : 2600 Cost : 0.67255086
STEP : 2800 Cost : 0.672549
STEP : 3000 Cost : 0.6725471
STEP : 3200 Cost : 0.6725455
STEP : 3400 Cost : 0.67254454
STEP : 3600 Cost : 0.67254364
STEP : 3800 Cost : 0.6725433
STEP : 4000 Cost : 0.672543
STEP : 4200 Cost : 0.6725427
STEP : 4400 Cost : 0.6725418
STEP : 4600 Cost : 0.6725409
STEP : 4800 Cost : 0.6725406
STEP : 5000 Cost : 0.6725404
STEP : 5200 Cost : 0.6725402
STEP : 5400 Cost : 0.6725399
STEP : 5600 Cost : 0.67253965
STEP : 5800 Cost : 0.67253923
STEP : 6000 Cost : 0.67253894
STEP : 6200 Cost : 0.67253864
STEP : 6400 Cost : 0.6725383
STEP : 6600 Cost : 0.67

### Using Model

In [38]:
sess = tf.Session()

# Model Restore
saver.restore(sess, save_path)
    
# Input Values
weight = input("Weight : ")
height = input("Height : ")
old = input("How Old : ")

# Model Testing
print("Model Test...")
result = sess.run(hypothesis, feed_dict={train_x: [[old, height, weight]]})[0][0]

print(result)

if round(result) == 1:
    print("Gold Medal")

elif round(result) == 2:
    print("Silver Medal")
    
elif round(result) == 3:
    print("Bronze Medal")

else:
    print("No Medal")

print("Finished")
    

INFO:tensorflow:Restoring parameters from ./checkpoints/olympic_nn_model.ckpt
Weight : 75
Height : 182
How Old : 31
Model Test...
1.9993827
Silver Medal
Finished
